In [2]:
import pandas as pd
import time
import json
import datetime
import pprint
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# global str
# string= "Currently 27% busy, usually 33% busy."
import re
getPlaceIDURL='https://googlemaps.github.io/js-samples/dist/samples/places-placeid-finder/index.html'
###############################################
#loading URL_list
df = pd.read_csv ('TimhortonsURL.csv')
# print (df)
print(df.info())
df=df.drop_duplicates(subset = ["URL"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     175 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 170
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     69 non-null     object
dtypes: object(1)
memory usage: 1.1+ KB
None


In [3]:
url_list= df["URL"].values
# print(url_list)
#####################################################################
#opernWebdriver
browser = webdriver.Firefox()
######################################################################
weekDay=["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
# a Python object (dict):

########################################### prepare document to collect the data
for url in url_list:
    #################################
    document = {}
    document["GoogleID"]="213424325432sdfdg"
    document["currently-occupied-by"]= 112
    document["name"]="Tim Hortons"
    document["address"]="195 Westney Rd S, Ajax, ON L1S 2C9"

    for i in weekDay:
        document[i]={}
    #################################################
    browser.get(url)
    browser.execute_script("window.scrollTo(0, 20)") 
    #############################################################
    #get the addresss
    time.sleep(4)
    data_item_id=browser.find_elements(By.XPATH,"//button[@data-item-id='address']")
     
    for div in  data_item_id:
        if div.get_attribute('aria-label'):
    #             print(div.get_attribute('aria-label'))
                string=div.get_attribute('aria-label')
    #             print(string[string.find(": ")+2:])
                string= string[string.find(": ")+2:]+', Canada'
                document["address"]=string
                print(string)
    #############################################################
    div_tag =browser.find_elements(By.TAG_NAME , "div")
   
    weekIndex=-1
    lastTime="p.m"
    for div in  div_tag:
        if div.get_attribute('aria-label'):
            divString=div.get_attribute('aria-label')
            if divString:
                ####################################
                if "a.m" in divString and "p.m" in lastTime:
                    weekIndex=weekIndex+1 ##a new day    
                #################################### case of currently time         
                if "Currently" in divString:
                    busy= re.findall(r'\d+',divString)[1]
                    now = datetime.now()
                    AMorPM=" a.m"
                    current_time = now.strftime("%H")
                    if int(current_time)>12:
                        current_time=int(current_time)%12
                        current_time=str(current_time)
                        AMorPM=" p.m"
                    theTime=current_time+AMorPM
                    document[weekDay[weekIndex]][theTime]=busy
                    lastTime=theTime
                if "a.m" in divString or "p.m" in divString:
#                     busy=int(divString[0:divString.find("%")])
                    busy=int(re.findall(r'\d+',divString)[0])
                    if "a.m" in divString:
                        AMorPM = " a.m"
                    else:
                        AMorPM = " p.m"
                    timepm= re.findall(r'\d+',divString)[1]
                    theTime= timepm+AMorPM
                    document[weekDay[weekIndex]][theTime]=busy
                    lastTime=theTime
#     pprint.pprint(document)

    #############################################
    #get PlaceID using the saved Address
    browser.get(getPlaceIDURL)
    input_text= browser.find_element(By.XPATH,"//input[@id='pac-input']")
    time.sleep(3)
    input_text.click()
    input_text.send_keys(document["address"])
    time.sleep(1)
    input_text.send_keys(Keys.ARROW_DOWN)
    input_text.send_keys(Keys.ENTER)
    time.sleep(1)

    document["GoogleID"]=browser.find_element(By.XPATH, "//span[@id='place-id']").text
    print(document["GoogleID"])
    #############################################
    df_data =pd.DataFrame([document])
    df_data.to_csv('data.csv', mode='a', index=False, header=False)

563 Sherbourne St, Toronto, ON M4X 1W7 , Canada
ChIJh4BBM1LL1IkR913aOlTE4yI
70 Gerrard St W, Toronto, ON M5G 1J5 , Canada
ChIJ8w8bD8o0K4gRsjdJOEgQwpI
175 Dundas St E, Toronto, ON M5A 1Z4 , Canada
ChIJQfeDDjbL1IkRlZp9TVjtAdE
663 Yonge St, Toronto, ON M4Y 1Z9 , Canada
ChIJQ-6yAbI0K4gRDmz8ZTibj7E
555 Yonge St, Toronto, ON M4Y 3A6 , Canada
ChIJ38culrM0K4gRfw5N7vAXNCc
175 Bloor St E, Toronto, ON M4W 3R8 , Canada
ChIJCxDi_Kw0K4gR1DQTKkGA4fY
250 Bloor St E, Toronto, ON M4W 1E5 , Canada
ChIJE1BgsKw0K4gRiKDGyRJndAc
1 Mt Pleasant Rd, Toronto, ON M4Y 2Y5 , Canada
ChIJ_ySW3lLL1IkROao24vam_NI
56 Wellesley St W Unit 104, Toronto, ON M5S 2S3 , Canada
ChIJrcUxMbE0K4gRSQzUGyiPhis
769 Yonge St, Toronto, ON M4W 2G4 , Canada
ChIJQ3Keia40K4gRNeyiWfGQfbE
589 King St E, Toronto, ON M5V 1M1 , Canada
ChIJmdP9smrL1IkRJf4aKsmzt6A
10 Dundas St E Suite 104, Toronto, ON M5B 2G8 , Canada
EicxMCBEdW5kYXMgU3QgRSwgVG9yb250bywgT04gTTVCLCBDYW5hZGEiMBIuChQKEgmpcRHUyjQriBFNwvrpB5DKYhAKKhQKEglxTn4vcMvUiRH_juQHFyDNtw
595 Bay

In [ ]:
import pandas as pd
import time
import json
import datetime
import pprint
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# global str
# string= "Currently 27% busy, usually 33% busy."
import re
getPlaceIDURL='https://googlemaps.github.io/js-samples/dist/samples/places-placeid-finder/index.html'
###############################################
#loading URL_list
df = pd.read_csv ('TimhortonsURL.csv')
# print (df)
url_list= df["URL"].values
# print(url_list)
#####################################################################
#opernWebdriver
browser = webdriver.Firefox()
######################################################################
weekDay=["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
# a Python object (dict):

########################################### prepare document to collect the data
document = {}
document["GoogleID"]="213424325432sdfdg"
document["currently-occupied-by"]= 112
document["name"]="Tim Hortons"
document["address"]="195 Westney Rd S, Ajax, ON L1S 2C9"

for i in weekDay:
    document[i]={}
#################################################
browser.get('https://www.google.ca/maps/place/Tim+Hortons/data=!4m5!3m4!1s0x89d51e6c109c7bb9:0xe862cf9d09617ef7!8m2!3d43.8861392!4d-78.945305?authuser=0&hl=en&rclk=1')
browser.execute_script("window.scrollTo(0, 20)") 
#############################################################
#get the addresss
data_item_id=browser.find_elements(By.XPATH,"//button[@data-item-id='address']")

for div in  data_item_id:
    if div.get_attribute('aria-label'):
#             print(div.get_attribute('aria-label'))
            string=div.get_attribute('aria-label')
#             print(string[string.find(": ")+2:])
            string= string[string.find(": ")+2:]+', Canada'
            document["address"]=string
            print(string)

#############################################################
div_tag =browser.find_elements(By.TAG_NAME , "div")
weekIndex=-1
lastTime="p.m"
for div in  div_tag:
    if div.get_attribute('aria-label'):
        divString=div.get_attribute('aria-label')
        if divString:
            ####################################
            if "a.m" in divString and "p.m" in lastTime:
                weekIndex=weekIndex+1 ##a new day    
            #################################### case of currently time         
            if "Currently" in divString:
                busy= re.findall(r'\d+',divString)[1]
                now = datetime.now()
                AMorPM=" a.m"
                current_time = now.strftime("%H")
                if int(current_time)>12:
                    current_time=int(current_time)%12
                    current_time=str(current_time)
                    AMorPM=" p.m"
                theTime=current_time+AMorPM
                document[weekDay[weekIndex]][theTime]=busy
                lastTime=theTime
            if "a.m" in divString or "p.m" in divString:
                busy=int(divString[0:divString.find("%")])
                if "a.m" in divString:
                    AMorPM = " a.m"
                else:
                    AMorPM = " p.m"
                timepm= re.findall(r'\d+',divString)[1]
                theTime= timepm+AMorPM
                document[weekDay[weekIndex]][theTime]=busy
                lastTime=theTime
pprint.pprint(document)

#############################################
#get PlaceID using the saved Address
browser.get(getPlaceIDURL)
input_text= browser.find_element(By.XPATH,"//input[@id='pac-input']")
print(input_text.get_attribute('placeholder'))
time.sleep(5)
input_text.click()
input_text.send_keys(document["address"])
time.sleep(2)
input_text.send_keys(Keys.ARROW_DOWN)
input_text.send_keys(Keys.ENTER)
time.sleep(2)

document["GoogleID"]=browser.find_element(By.XPATH, "//span[@id='place-id']").text
print(document["GoogleID"])
#############################################

df_data =pd.DataFrame([document])
df_data.to_csv('data.csv', mode='a', index=False, header=False)

In [2]:
import re
vv= "19% busy at 3 p.m.."
nn= int(re.findall(r'\d+',vv)[0])
print(nn)
print(type(nn))

19
<class 'int'>
